# Plano de Teste

### Testes na camada trusted para os dados sobre de geolocalização.
Testes realizados após transformar os dataset de formato shapefile em csv


In [1]:
#importando bibliotecas necessárias para os testes
import pandas as pd
import great_expectations as gx 

#Criar o df tax_df com as informacoes doa arquivo us_tax_data_2016.csv
path ='./dados/validacoes_trusted/trusted_geo_brasil.csv'
df_geo = pd.read_csv(path, delimiter=';', encoding="ansi")

In [2]:
# verificacao do layout do dataset
print(df_geo.head(3))


    CD_MUN                 NM_MUN SIGLA_UF distancia_   
0  1100015  Alta Floresta D'Oeste       RO        7km  \
1  1100023              Ariquemes       RO        4km   
2  1100031                 Cabixi       RO        1km   

                                            geometry  
0  POLYGON ((-62.0080637729999 -12.1337851349999,...  
1  POLYGON ((-63.179325881 -10.139244048, -63.177...  
2  POLYGON ((-60.524079765 -13.32137253, -60.3716...  


Crie uma expectation para utilizar no great expectation para relizações de algumas validações:

In [3]:
df_gx = gx.dataset.PandasDataset(df_geo)
df_gx.head(3)

,CD_MUN,NM_MUN,SIGLA_UF,distancia_,geometry
0,1100015,Alta Floresta D'Oeste,RO,7km,"POLYGON ((-62.0080637729999 -12.1337851349999,..."
1,1100023,Ariquemes,RO,4km,"POLYGON ((-63.179325881 -10.139244048, -63.177..."
2,1100031,Cabixi,RO,1km,"POLYGON ((-60.524079765 -13.32137253, -60.3716..."


``1`` Verifica se o código município (CD_MUN) contém apenas valores únicos.

Usar a função ``expect_column_values_to_be_unique`` para definir essa expectativa

In [4]:
validation_result = df_gx.expect_column_values_to_be_unique(column = 'CD_MUN')
validation_result

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5572,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  }
}

In [5]:
if validation_result['success']:
    print('A coluna de código de município contem apenas valores únicos')
else:
    print('A coluna código de município possui valores duplicados')

A coluna de código de município contem apenas valores únicos


``2`` Validação de valores nulos em uma coluna


In [6]:
validation_result = df_gx.expect_column_values_to_not_be_null(column= 'CD_MUN')
validation_result

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5572,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  }
}

In [7]:
if validation_result['success']:
    print('A coluna código do município não possui valores nulos')
else:
    print('A coluna código do município possui valores nulos, verificar a necessidade de realizar tratamento no campo')

A coluna código do município não possui valores nulos


``3`` Validação de siglas de UF incorretos

In [8]:
# Lista de siglas de UF válidas no Brasil
uf_validas = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA',
              'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN',
              'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

# Filtrar valores de 'uf' que não estão na lista de UF válidas
uf_invalida = df_geo[~df_geo['SIGLA_UF'].isin(uf_validas)]

if uf_invalida.empty:
    print('A coluna sigla UF não possui valor(es) incorreto(s) na coluna SIGLA_UF.')
else:
    print("Existe(m) valor(es) incorreto(s) na coluna 'uf':")
    print(uf_invalida['SIGLA_UF'].unique())


A coluna sigla UF não possui valor(es) incorreto(s) na coluna SIGLA_UF.


In [9]:
uf_invalida_count = uf_invalida['SIGLA_UF'].count()
print("Quantidade de Siglas de estado incorreta(s):", uf_invalida_count)


Quantidade de Siglas de estado incorreta(s): 0


``4`` Verifica se existe(m) distância que não esteja(m) em km.

In [10]:
# Verificar se não há valores na coluna 'distancia_' que contenham a palavra "km"
distancia_sem_km = df_geo[~df_geo['distancia_'].str.contains('km', case=False, na=False)]

if distancia_sem_km.empty:
    print("Todos os valores na coluna distancia_ contêm a palavra 'km'.")
else:
    print("Existem valores na coluna distancia_ que não contêm a palavra km:")
    print('Valor(es) incorreto(s):')
    print(distancia_sem_km['distancia_'].to_string(index=False))


Todos os valores na coluna distancia_ contêm a palavra 'km'.
